# Gestion des listes de signaux stockés au format HDF5
"hdf5set" contient essentiellement une classe `Selector` permettant de manipuler et d'accéder facilement à des signaux stockés dans un fichier HDF5.

Chaque signal est un pandas.DataFrame `df` dont le nom est tocké dans `df.index.name`.
Ces noms sont indépendants des noms d'enregistrement dans le fichier mais il est préférable de conserver les mêmes.

Il est préférable d'avoir stocké ainsi des signaux ayant les mêmes noms de colonnes. Les unités de chaque colonne si elles existent sont stockées entre crochets après le nom de la variable.

Une telle liste sera appelée "liste d'opérations" ou OPSET.

In [1]:
from importlib import reload
import numpy as np
import pandas as pd
import tabata.hdf5set as hs

## 1/ Visualisation des données.

La classe `Selector` permet d'accéder aux signaux, la fonction `Selector.plot()` offre une interface graphique sympatique pour naviguer au sein de la liste.

In [19]:
reload(hs)
storename = "data/AFL1EB.h5"
S = hs.Selector(storename)

In [21]:
S.plot2()

In [18]:
S.sel_instants

{}

Les données de ce jeu d'exemple représentent des mesures simulées de vols d'un avion.
    
* ALT[m] : l'altitude de l'avion en mètres.
* Tisa[K] : la température standard en Kelvin.
* TAS[m/s] : La vitesse air de l'avion (Total Air Speed) en mètre par seconde.
* Vz[m/s] : la vitesse de montée en mètre par seconde.
* Masse[kg] : la masse de l'avion en kilogramme.
* F[N] : la poussée en Newton.

L'enregistrement courant peut être récupéré facilement.

In [ ]:
S.df

On peut directement demander l'affichage spécifique d'un des signaux.

In [ ]:
S.plot(sigpos=21,colname="Vz[m/s]")

Cette demande peut aussi se faire dès l'instanciation.

In [ ]:
S = hs.Selector(storename,sigpos=5,colname="TAS[m/s]")
S.plot()

## Itérations
Il est possible d'itérer sur la liste de signaux. L'itérateur admet deux paramètres optionnels:  `first` et `last`. Attention l'indexation commence à 1.

L'itération met à jour le signal courant du sélecteur.

In [ ]:
n=0
for df in S.iterator(first=3,last=4):
    t0 = df.index[0]
    print("{:2d} : {:10s} --> {}".format(n,df.index.name,t0))
    n = n+1
    
print('Boucle n°1 :',S.df.index.name)

n=0
for df in S.iterator(first=3,last=8):
    t0 = df.index[0]
    print("{:2d} : {:10s}  --> {}".format(n,df.index.name,t0))
    n = n+1
    
print('Boucle n°1 :',S.df.index.name)

## Création d'un nouvel opset (liste d'opération)
À partir d'une première liste d'opérations il est possible d'en créer une nouvelle assez facilement à l'aide du sélecteur.

Dans notre exemple on suppose qu'il existe des vols mal enregistrés, on va les supprimer, par ailleurs il y a un problème de changement de date qui a abimé l'index temporel (à la seconde ici) ce que l'on va corriger en créant un jeu de données propres.

In [ ]:
reload(hs)
S = hs.Selector(storename)

La fonction `put(df,record)` permet une modification rapide d'un enregistrement. Par défaut, si aucun nom d'enregistrement n'est donné en second paramètre, on regarde si `df.index.name` existe. De façon équivalente, si l'index n'est pas nommé mais qu'un nom d'enregistrement sera passé, la fonction `put(df,record)` rajoutera le nom d'index. 

In [ ]:
# Construction d'un hdf5set sans données anormales.
cleanstore = "data/AFL1EB_C.h5"
Sc = hs.Selector(cleanstore)

for df in S.iterator():
    if max(df["F[N]"])>0: # Les données ont été bien enregistrées.
        x = df.index
        t = (x-x[0]).total_seconds()
        dt = np.diff(t)
        i = np.argwhere(dt != dt[1]) # on détecte un problème.
        if len(i)>0:
            name = df.index.name
            df.index = pd.date_range(x[0],periods=len(df),freq=x[1]-x[0]) # détruit le nom
            df.index.name = name # Ne pas oublier de récupérer le nom.
            Sc.put(df) # Ici on aurait pu mettre name.

La fonction `current_record()` est un racourci pour obtenir le nom de l'enregistrement courant.

In [ ]:
for df in Sc.iterator(1,4):
    print(Sc.current_record(), ':', df.index.name)

## Modification du dataset
Dans une itération, vu que l'enregistrement courrant est celui qui est renvoyé, si on ne précise pas de nouveau nom d'enregistrement, c'est l'enregistrement courant qui est modifié par `put()`.

Une fonction intéressante de la fonction `plot` est la possibilité d'afficher en surimpression une partie du signal. Pour cela il suffit de rajouter une variable booléenne aux signaux qui indique quelles partie du signal identifier.

Dans cet exemple nous essayons d'extraire la croisière.

In [ ]:
for df in Sc.iterator():
    mx = max(df["ALT[m]"])
    df["CR"] = (df["ALT[m]"]>mx-2000) & (abs(df["Vz[m/s]"])<1)
    Sc.put(df)

In [ ]:
reload(hs)
Sc = hs.Selector(cleanstore)

In [ ]:
Sc.plot("CR")

### Quelques éléments techniques de la fonction `plot`.

#### Gestion des signaux multivariés
La classe `Selector` permet de gérer des données stockées sous la forme d'une liste d'observations temporelles. Chaque observation est un signal multivarié indexé par le temps. Cette liste doit être stockée dans un fichier au format HDF5 où chaque enregistrement est une observations.

* Les observations sont lues dans l'ordre alphabétique, par un itérateur `Ìterator`.
* Chaque signal (observation) est synchrones : toutes les colonnes ont la même longuer et le signal est stocké sous la forme d'un DataFrame pandas (`df`).
* Le nom du signal est stocké dans le nom de l'index (`df.index.name`), ce qui permet de nommer les enregistrements indépendamment de leur nom.
* Le nom de chaque colonne correspond au nom de la variable suivi entre crochets de son unité.

#### Affichage des données.
La fonction `Selector.plot()` sert à afficher les données de manière interactive à laide de Plotly au sein d'un notebook Python. Pour cela l'import du package hdf5set exécute la fonction `init_notebook_mode` de Plotly permettant d'interagir au sein du notebook à l'aide de `iplot()`.

Pour naviguer entre les signaux, le Selector utilise des gadgets (boutons, scrollbar, menu) pour passer d'un enregistrement à un autre. Ces gadgets sont issus du packae ipywidgets qui est compatible avec Plotly. Le code de la fonction `Selector.plot()` est un exemple ssez parlant de l'exploitation de gadgets interactifs au sein d'un notebook Jupyter.

* Un menu permet de sélectionner une colonne à afficher.
* Deux boutons (Previous et Next) passent d'un signal au suivant dans l'ordre des enregistrements.
* Une scrollbar verticale sur la droite rappelle l'enregistrement en cours de visualisation et permet aussi de se déplacer aléatoirement entre les enregistrements.

La liste ordonnée des enregistrements est sauvegardée dans une variables `.records`, le signal en cours d'affichage est stocké dans `.sigpos` (en commençant par le n°1), et le nom de la variable à afficher est conservé dans `.colname`.

#### Fonctionnement de l'interactivité
L'interactivité est donnée par l'appel

    out = widgets.interactive(update_plot, colname=wd, sigpos=ws)
  
`wd` est un pointeur vers le menu Dropdown contenat la liste des variables du signal en cours d'affichage et `ws` un pointeur vers la scrollbar.

       wd = widgets.Dropdown(options=df.columns, description="Variable :")
       ws = widgets.IntSlider(value=1, min=1, max=nbmax, step=-1,
                               orientation='vertical',
                               description='Record',
                               layout=widgets.Layout(height='400px'))
                               
Cette dernière est en mode 'vertical' et sa hauteur est fixée à 400 pixels. Le nombre maximum d'enregistrements a été stocké dans la variable `nbmax`. La fonction `interactive` lie le contenu des gadgets au variables locales `colanme` et `sigpos`qui sont passées à la fonction locale `update_plot()` qui fait le travail d'affichage en mettant à jour les éléments définis par le premier affichage initila à l'apple de `.plot()`.

Si la scrollbar apparait sur la droite de l'affichage c'est parce que l'on a juxtaposé deux HBox dans une VBox que l'on revoie comme retour de `.plot()` :

        boxes = widgets.VBox([widgets.HBox([wd, wbp, wbn]), widgets.HBox([
            f, ws])])
        return boxes

La première HBox contient le menu et les deux boutons, et juste en dessous nous plaçons une seconde Hbox qui contient d'abbord la dfigure `f` créée initialement et à sa droite la scrollbar verticale.

    f = go.FigureWidget(data, layout)
    
L'interaction des boutons est assez simple : on le lie à une fonction callback locale `wb_on_click()` que l'on associe à chaque widget par l'appel de `.on_click()`. En fait cette fonction se contente de modifier la valeur de la scrollbar ce qui entrainera automatiquement par interactivité une modification de l'afficahe comme si on avait directement cliqué sur la scrollbar.

#### Mise en évidence d'une partie des données
Le Selector bénéficie de deux fonction d'affichage supplémentaire. La première consiste à mettre en évidence par un coloriage en rouge une partie du signal.
Pour cela il faut qu'une variable booléenne soit présente dans le signal multivarié comme une colonne particulière. Le nom de cette variable peut être passé en argument `phase`de `.plot(phase)` et les points 'True' de cette 'phase' seront affichés en rouge.

#### Sélection d'instants
La seconde fonctionnalité originale consiste en la possibilité de sélectionner des instant en cliquant sur la courbe affichée. Une barre verticale va alors apparaitre pour mettre en évidence ces instants graphiquement. On peut sélectionner un seul instant par signal, mais on peut le faire à partir de n'importe quelle variable.
La liste des couples (variable, instant) est stockée dans la variable `sel_instants` du Selector et peut être récupérée par un simple appel.

    S = Selector.plot()
    instants = S.sel_instants
    
Les signaux observés lors de la sélection peuvent être facilement identifiés par :

    op = S.op_viewed
    
L'interactivité de la sélection d'instant est gérée par la fonction callbck locale `selection_fn` qui est ensuite liée à la trace de la courbe bleue par un appel à `.on_click()`.

Les enregistrements vus et isntants stockés peuvent être supprimés par la fonction `clean_selection()`.

In [ ]:
Sc.plot()

In [ ]:
Sc.sel_instants

In [ ]:
Sc.op_viewed